In [1]:
import sys
sys.path.append(r'C:\Users\maxfu\OneDrive\Documentos\Master\Maximaster\Year 2 - Period 1\Project\Double Pendulum\Double Pendulum')
import numpy as np
from scinet import *
import scinet.pendulum as edo
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.feature_selection import mutual_info_regression
from scipy.integrate import odeint

Instructions for updating:
non-resource variables are not supported in the long term


# Data loading and testing #

Once the program has been trained, we can check the output results.

In [2]:
saved = nn.Network.from_saved('Double Pendulum data')

{'latent_size': 4, 'input_1_size': 50, 'input_2_size': 50, 'time_size': 1, 'output_1_size': 50, 'output_2_size': 50, 'encoder_num_units': [500, 100], 'decoder_num_units': [100, 100], 'tot_epochs': 1000, 'name': 'Unnamed', 'load_file': 'Double Pendulum data'}
INFO:tensorflow:Restoring parameters from C:\Users\maxfu\OneDrive\Documentos\Master\Maximaster\Year 2 - Period 1\Project\Double pendulum\Double Pendulum\tf_save/Double Pendulum data.ckpt
Loaded network from file Double Pendulum data


We define the data generation functions for the predictions. 

In [9]:
def derivatives(sol,t,L1,L2,m1,m2):
    
    g=9.81
    Theta_1,Theta_1_dot,Theta_2,Theta_2_dot = sol
    
    C1=np.cos(Theta_1-Theta_2)
    C2=np.sin(Theta_1-Theta_2)
    
    # The first derivative of Theta is named Z, and so the second derivative is named Z_dot
    
    Z_1=Theta_1_dot
    Z_2=Theta_2_dot
    Z_1_dot=(m2*g*np.sin(Theta_2)*C1-m2*C2*(L1*Z_1**2*C1+L2*Z_2**2)-(m1+m2)*g*np.sin(Theta_1))/(L1*(m1+m2*C2**2))
    Z_2_dot=((m1+m2)*(L1*Z_1**2*C2-g*np.sin(Theta_2)+g*np.sin(Theta_1)*C1)+m2*L2*Z_2**2*C2*C1)/(L2*(m1+m2*C2**2))
    
    return Z_1,Z_1_dot,Z_2,Z_2_dot

################################################################################################

def gen_input(Theta_1_ini,Theta_2_ini,L1,L2,m1,m2, t_predicted):
    t_in = np.linspace(0, 5, 50)
    sol_i = np.array([Theta_1_ini,0,Theta_2_ini,0])
    sol = odeint(derivatives, sol_i, t_in, args=(L1,L2,m1,m2))
    Theta_1,Theta_2=sol[:,0],sol[:,2]
    sol_res = np.reshape(t_predicted, (-1, 1))
    out = sol_res #dummy filler
    return [Theta_1,Theta_2, sol_res, out]

In [10]:
blue_color='#033cff'
orange_color='#ff7700'

In [16]:
t_predicted = np.linspace(0, 10, 250)
sol_predicted = saved.run(gen_input(0,np.pi/8,1,1,1,1, t_predicted),saved.output()).ravel()

TypeError: 'Tensor' object is not callable

In [17]:
def pendulum_prediction(net, L1, L2, m1, m2):    
    t_given = np.linspace(0, 10, 250)
    t_predicted = np.linspace(0, 10, 250)
    ################################################
    sol_i_given = np.array([0,0,np.pi/8,0])
    sol_given = odeint(derivatives, sol_i_given, t_given, args=(L1,L2,m1,m2))
    Theta_1_given,Theta_2_given=sol_given[:,0],sol_given[:,2]
    x1 = L1 * np.sin(Theta_1_given)
    y1 = -L1 * np.cos(Theta_1_given)
    x2 = L1 * np.sin(Theta_1_given) + L2 * np.sin(Theta_2_given)
    y2 = -L1 * np.cos(Theta_1_given) - L2 * np.cos(Theta_2_given)
    ################################################
    sol_predicted = net.run(gen_input(0,np.pi/8,L1,L2,m1,m2, t_predicted), net.output).ravel()
    print((net.output).shape)
    fig = plt.figure(figsize=(3.4, 2.1))
    ax = fig.add_subplot(111)
    ax.plot(t_given, x1, color=orange_color, label='Theta 1')
    ax.plot(t_given, Theta_1_given,'--', color=blue_color, label='Theta 2')
    
    ax.plot(t_predicted, sol_precicted, '--', color=blue_color, label='Predicted time evolution')
    ax.set_xlabel(r'$t$ [$s$]')
    ax.set_ylabel(r'$x$ [$m$]')
    handles, labels = ax.get_legend_handles_labels()
    lgd=ax.legend(handles, labels,loc='upper center', bbox_to_anchor=(0.6, 1.3), shadow=True, ncol=1)
    fig.tight_layout()
    return fig

In [18]:
pendulum_prediction(saved, 1.5, 2.2, 1, 2);

ValueError: Cannot feed value of shape (50,) for Tensor input1:0, which has shape (?, 50)